In [1]:
import torch
from RealESRGAN import RealESRGAN
import os
import argparse
import itertools
import numpy as np
from tqdm import tqdm
import torch.optim as optim
from torchvision.utils import save_image
from torch import Tensor
from torchvision import transforms
from torch.utils.data import DataLoader,Dataset
from glob import glob
from torch.utils.data.distributed import DistributedSampler
import random
from PIL import Image
import torchvision
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
from torchvision import models
from matplotlib import pyplot as plt
import nibabel as nib
from cyclegan_3d.base_model import BaseModel
from cyclegan_3d.cycle_gan_model import CycleGANModel
from cyclegan_3d.networks3D import define_G, define_D
#-*- coding:utf-8 -*-
from med_ddpm.diffusion_model.trainer import GaussianDiffusion, num_to_groups
from med_ddpm.diffusion_model.trainer import GaussianDiffusion, Trainer
from med_ddpm.diffusion_model.unet import create_model
from torchvision.transforms import Compose, Lambda
from med_ddpm.utils.dtypes import LabelEnum
import torchio as tio


print(f"GPUs used:\t{torch.cuda.device_count()}")
device = torch.device("cuda",1)
print(f"Device:\t\t{device}")
import pytorch_model_summary as tms
import random
import cv2
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

2025-05-28 17:36:57.430868: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-28 17:36:57.446376: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-28 17:36:57.450721: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-28 17:36:57.463791: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-28 17:36:58.164246: W tensorflow/compiler/tf2

APEX: OFF
GPUs used:	8
Device:		cuda:1



<font size="6">lschemic_CT & SuperResolution</font>

In [9]:
params = {
    "with_condition": True,
    "inputfolder": "../../data/registration_data/registration_DWI/",
    "batchsize": 1,
    "epochs": 10000,
    "input_size": 128,
    "depth_size": 64,
    "num_channels": 64,
    "num_res_blocks": 1,
    "timesteps": 250,
    "save_and_sample_every": 10,
    "model_save_path": "../../model/med_ddpm_translation/dwi2ct/",
    "resume_weight": "../../model/med_ddpm/translation/dwi2ct_1.pt"
}
device = "cuda:1" if torch.cuda.is_available() else "cpu"
print("Device: ", device)


Device:  cuda:1


In [10]:
def save_sample(diffusion, sample_shape, device,temp, condition_tensors=None):
    diffusion.eval()
    transform = Compose([
    Lambda(lambda t: torch.tensor(t).float()),
    Lambda(lambda t: t.unsqueeze(0))
        ])
    batchsize=8
    # 샘플 생성
    condition_tensors1= transform(condition_tensors.get_fdata()).unsqueeze(0).repeat(batchsize, 1, 1, 1, 1)
    max_file_size_kb = 3072
    file_saved = False
    temp1_img=0
    saa=1090000
    for j in range(1):
        samples = diffusion.sample(batch_size=batchsize, condition_tensors=condition_tensors1.to(device))
        samples = samples.cpu()
        for i in range(batchsize):
            temp_img=samples[i][0].numpy()
            nifti_img = nib.Nifti1Image(temp_img, affine=np.eye(4))
            nib.save(nifti_img, temp)
            file_size_kb = os.path.getsize(temp) / 1024
            if file_size_kb <= max_file_size_kb:
                    return temp_img
            else:
                if saa> file_size_kb:
                    saa=file_size_kb
                    temp1_img= temp_img

                continue
        

    return temp1_img

def to_numpy_img(tensor):
    tensor = tensor.detach().cpu().clamp(0, 1)  # ensure valid range
    return tensor.permute(1, 2, 0).numpy()
tf=transforms.ToTensor()
topilimage = torchvision.transforms.ToPILImage()
dwi_sr_model = RealESRGAN(device, scale=2).model.to(device)
dwi_sr_model.load_state_dict(torch.load('../../model/ESRGAN/DWI/ckpt_45_checkpoint.pt',map_location=device))
ct_sr_model = RealESRGAN(device, scale=2).model.to(device)
ct_sr_model.load_state_dict(torch.load('../../model/ESRGAN/CT/ckpt_44_checkpoint.pt',map_location=device))
tr_model = create_model(params["input_size"], params["num_channels"], params["num_res_blocks"], in_channels=2, out_channels=1).to(device)
diffusion = GaussianDiffusion(
    tr_model,
    image_size = params["input_size"],
    depth_size = params["depth_size"],
    timesteps = 250,   # number of steps
    loss_type = 'L1', 
    with_condition=True,
).to(device)
diffusion.load_state_dict(torch.load(params["resume_weight"],map_location=device))
print("Model Loaded!")
diffusion.eval()
from types import SimpleNamespace

data_path='../../result/generator/Ischemic_DWI/'
save_path='../../result/generator_sr/'
nii_list=glob(data_path+'*.nii.gz')
nii_mask_list=[f.replace('/Ischemic_DWI','/Ischemic_mask') for f in nii_list]
step=0
for i in tqdm(range(len(nii_list))):
    nii_img=nib.load(nii_list[i])
    nii_img_data=(nii_img.get_fdata()+1.)/2.
    nii_img_mask=nib.load(nii_mask_list[i])
    nii_img_mask_data=nii_img_mask.get_fdata()
    step+=1
    folder_name=str(step).zfill(6)
    tensor_A=tf(nii_img.get_fdata()).float().to(device)
    a=tensor_A.permute(1, 2, 0).unsqueeze(0).unsqueeze(0)
    with torch.no_grad():
        fake_CT = save_sample(diffusion=diffusion,
            sample_shape=(1, diffusion.channels, diffusion.depth_size, diffusion.image_size, diffusion.image_size),
            device=device,
            condition_tensors=nii_img,
            temp='../../result/generator_sr/temp/dwi2ct.nii.gz')  # Clamp to 0~1 range
        fake_CT[:, :5, :]=-1
        fake_CT[:, -5:, :]=-1
        fake_CT[:, :, :5]=-1
        fake_CT[:, :, -5:]=-1
        fake_CT=(fake_CT+1.)/2

        fake_CT=fake_CT.squeeze()
    dwi_slide=[]
    ct_slide=[]
    mask_slide=[]
    for j in range(len(nii_img_data)):  # len(nii_img_data)
        image=tf(topilimage(nii_img_data[j]).convert('RGB')).float().to(device)
        image=image.unsqueeze(0)
        ct_image=tf(topilimage(fake_CT[j]).convert('RGB')).float().to(device).unsqueeze(0)
        sr_mask=cv2.resize(nii_img_mask_data[j],(256,256))
        sr_mask=np.where(sr_mask==2,1,0)*255
    
        mask_slide.append(sr_mask)
        with torch.no_grad():
            sr = dwi_sr_model(image)
            sr = sr.squeeze(0).cpu().numpy()
            sr_ct=F.sigmoid(ct_sr_model(ct_image))
            sr_ct = sr_ct.squeeze(0).cpu().numpy()
            dwi_slide.append(sr[0])
            ct_slide.append(sr_ct[0])
        
    dwi_slide=np.array(dwi_slide)
    # min-max 정규화
    dwi_slide = (dwi_slide - dwi_slide.min()) / (dwi_slide.max() - dwi_slide.min())
    dwi_slide=dwi_slide*255
    dwi_slide = dwi_slide.astype(np.uint8)
    ct_slide=np.array(ct_slide)

    ct_slide = (ct_slide - ct_slide.min()) / (ct_slide.max() - ct_slide.min())
    ct_slide=ct_slide*255
    ct_slide = ct_slide.astype(np.uint8)
    mask_slide=np.array(mask_slide).astype(np.uint8)
    create_dir(save_path+'Ischemic_DWI/'+folder_name)
    create_dir(save_path+'Ischemic_CT/'+folder_name)
    create_dir(save_path+'Ischemic_mask/'+folder_name)
    for j in range(len(nii_img_data)):
        Image.fromarray(dwi_slide[j]).save(save_path+'Ischemic_DWI/'+folder_name+'/'+str(j).zfill(6)+'.png')
        Image.fromarray(ct_slide[j]).save(save_path+'Ischemic_CT/'+folder_name+'/'+str(j).zfill(6)+'.png')
        Image.fromarray(mask_slide[j]).save(save_path+'Ischemic_mask/'+folder_name+'/'+str(j).zfill(6)+'.png')

Model Loaded!


  0%|          | 8/6479 [15:49<213:22:41, 118.71s/it]


KeyboardInterrupt: 

In [14]:
ct_image

tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]], device='cuda:1')

In [5]:
fake_CT.max()

1.0

<font size="6">Normal CT & SuperResolution</font>

In [ ]:
ct_sr_model = RealESRGAN(device, scale=2).model.to(device)
ct_sr_model.load_state_dict(torch.load('../../model/ESRGAN/CT/ckpt_12_checkpoint.pt',map_location=device))

In [ ]:
def save_sample(diffusion, sample_shape, device,temp, condition_tensors=None):
    diffusion.eval()
    transform = Compose([
    Lambda(lambda t: torch.tensor(t).float()),
    Lambda(lambda t: t.unsqueeze(0))
        ])
    batchsize=5
    # 샘플 생성
    condition_tensors1= transform(condition_tensors.get_fdata()).unsqueeze(0).repeat(batchsize, 1, 1, 1, 1)
    max_file_size_kb = 3072
    file_saved = False
    temp1_img=0
    saa=1090000
    for j in range(1):
        samples = diffusion.sample(batch_size=batchsize, condition_tensors=condition_tensors1.to(device))
        samples = samples.cpu()
        for i in range(batchsize):
            temp_img=samples[i][0].numpy()
            nifti_img = nib.Nifti1Image(temp_img, affine=np.eye(4))
            nib.save(nifti_img, temp)
            file_size_kb = os.path.getsize(temp) / 1024
            if file_size_kb <= max_file_size_kb:
                    return temp_img
            else:
                if saa> file_size_kb:
                    saa=file_size_kb
                    temp1_img= temp_img
                os.remove(temp)  # 너무 크면 삭제하고 재시도
                continue
        

    return temp1_img

def to_numpy_img(tensor):
    tensor = tensor.detach().cpu().clamp(0, 1)  # ensure valid range
    return tensor.permute(1, 2, 0).numpy()
tf=transforms.ToTensor()
topilimage = torchvision.transforms.ToPILImage()
# dwi_sr_model = RealESRGAN(device, scale=2).model.to(device)
# dwi_sr_model.load_state_dict(torch.load('../../model/ESRGAN/SWI/ckpt_45_checkpoint.pt',map_location=device))
ct_sr_model = RealESRGAN(device, scale=2).model.to(device)
ct_sr_model.load_state_dict(torch.load('../../model/ESRGAN/CT/ckpt_12_checkpoint.pt',map_location=device))
tr_model = create_model(params["input_size"], params["num_channels"], params["num_res_blocks"], in_channels=2, out_channels=1).to(device)
diffusion = GaussianDiffusion(
    tr_model,
    image_size = params["input_size"],
    depth_size = params["depth_size"],
    timesteps = 250,   # number of steps
    loss_type = 'L1', 
    with_condition=True,
).to(device)
diffusion.load_state_dict(torch.load(params["resume_weight"],map_location=device))
print("Model Loaded!")
diffusion.eval()
from types import SimpleNamespace

data_path='../../result/generator/Normal_DWI/'
save_path='../../result/generator_sr/'
nii_list=glob(data_path+'*.nii.gz')
nii_mask_list=[f.replace('/Normal_DWI','/Normal_mask') for f in nii_list]
step=0
for i in tqdm(range(len(nii_list))):
    nii_img=nib.load(nii_list[i])
    nii_img_data=(nii_img.get_fdata()+1.)/2.
    nii_img_mask=nib.load(nii_mask_list[i])
    nii_img_mask_data=nii_img_mask.get_fdata()
    step+=1
    folder_name=str(step).zfill(6)
    tensor_A=tf(nii_img.get_fdata()).float().to(device)
    a=tensor_A.permute(1, 2, 0).unsqueeze(0).unsqueeze(0)
    # with torch.no_grad():
    #     fake_CT = save_sample(diffusion=diffusion,
    #         sample_shape=(1, diffusion.channels, diffusion.depth_size, diffusion.image_size, diffusion.image_size),
    #         device=device,
    #         condition_tensors=nii_img,
    #         temp='../../result/generator_sr/temp/dwi2ct.nii.gz')  # Clamp to 0~1 range
    #     fake_CT=(fake_CT+1.)/2
    #     fake_CT[:, :20, :]=0
    #     fake_CT[:, -20:, :]=0
    #     fake_CT[:, :, :20]=0
    #     fake_CT[:, :, -20:]=0
    #     fake_CT=fake_CT.squeeze()
    dwi_slide=[]
    ct_slide=[]
    mask_slide=[]
    for j in range(len(nii_img_data)):  # len(nii_img_data)
        image=tf(topilimage(nii_img_data[j]).convert('RGB')).float().to(device)
        image=image.unsqueeze(0)
        ct_image=tf(topilimage(fake_CT[j]).convert('RGB')).float().to(device).unsqueeze(0)
        sr_mask=cv2.resize(nii_img_mask_data[j],(256,256))
        sr_mask=np.where(sr_mask==2,1,0)*255
    
        mask_slide.append(sr_mask)
        with torch.no_grad():
            sr = dwi_sr_model(image)
            sr = sr.squeeze(0).cpu().numpy()
            sr_ct=ct_sr_model(ct_image)
            sr_ct = sr_ct.squeeze(0).cpu().numpy()
            dwi_slide.append(sr[0])
            ct_slide.append(sr_ct[0])
        
    dwi_slide=np.array(dwi_slide)
    # min-max 정규화
    dwi_slide = (dwi_slide - dwi_slide.min()) / (dwi_slide.max() - dwi_slide.min())
    dwi_slide=dwi_slide*255
    dwi_slide = dwi_slide.astype(np.uint8)
    ct_slide=np.array(ct_slide)

    ct_slide = (ct_slide - ct_slide.min()) / (ct_slide.max() - ct_slide.min())
    ct_slide=ct_slide*255
    ct_slide = ct_slide.astype(np.uint8)
    mask_slide=np.array(mask_slide).astype(np.uint8)
    create_dir(save_path+'Normal_DWI/'+folder_name)
    create_dir(save_path+'Normal_CT/'+folder_name)
    create_dir(save_path+'Normal_mask/'+folder_name)
    for j in range(len(nii_img_data)):
        Image.fromarray(dwi_slide[j]).save(save_path+'Normal_DWI/'+folder_name+'/'+str(j).zfill(6)+'.png')
        Image.fromarray(ct_slide[j]).save(save_path+'Normal_CT/'+folder_name+'/'+str(j).zfill(6)+'.png')
        Image.fromarray(mask_slide[j]).save(save_path+'Normal_mask/'+folder_name+'/'+str(j).zfill(6)+'.png')

Model Loaded!


  0%|          | 16/4419 [21:09<97:03:26, 79.36s/it]


KeyboardInterrupt: 

In [ ]:
from torchinfo import summary
summary(tr_model.netG_A, input_size=(1, 1, 64, 128, 128), device=opt.device)